<a href="https://colab.research.google.com/github/Viktory4121/MM/blob/main/%D0%9B%D0%A03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Лабораторная работа №3.
#### Клеточные конечные автоматы.
##### Вариант 11.

In [5]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

#### Входные параметры:

In [29]:
t = [i for i in range(1, 11)]

#### Генерация основной части таблицы состояний:

In [34]:
def generation_table_states():
  #table = [[0 for j in range(5)] for i in range(32)]
  table = pd.DataFrame(it.product([0,1], repeat=5))
  table['x(t;c1;c2)'] = [0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]
  table.columns = ['x(t-1;c1;c2)', 'x(t-1; c1-1;c2)', 'x(t-1;c1+1;c2)', 'x(t-1;c1;c2-1)', 'x(t-1;c1;c2+1)', 'x(t;c1;c2)']
  return table

#### Работа с таблицей состояний: